In [1]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
from xgboost import XGBRegressor
import plotly.express as px
from pathlib import Path
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [2]:
def upload_data():
    data_path = str(Path.cwd().parent) + "\\Data\\EPC\\Power Consumption Data.csv"
    
    df = pd.read_csv(data_path)
    
    df = df[df["real_consumption"] > 0]
    df = df[df['real_consumption'] <= df['real_consumption'].mean() + 4 * df['real_consumption'].std()]
    
    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values(by='time',ascending=True)
    
    return df

def data_metrics(data, real, predicted):

    y_true = data[real]
    y_pred = data[predicted]

    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)


    # MAE (Mean Absolute Error):
    # Lower values are better; good MAE depends on the scale of 'real_consumption'.
    # As a rule of thumb, MAE should be significantly smaller than the mean of the target variable.
    # Lower is better. Ideally, MAE should be much less than the average value of y_true.
    print(f"MAE: {mae:.4f}")

    # MSE (Mean Squared Error):
    # Similar to MAE but penalizes large errors more heavily. A smaller MSE is better.
    # Compare MSE to the variance of 'real_consumption' for context.
    # Lower is better. MSE should ideally be close to zero relative to the variance of y_true.
    print(f"MSE: {mse:.4f}")



    # RMSE (Root Mean Squared Error):
    # RMSE is the square root of MSE and is in the same units as 'real_consumption'.
    # A good RMSE is often close to the standard deviation of 'real_consumption'.
    # Lower is better. RMSE should be comparable to or less than the standard deviation of y_true."
    print(f"RMSE: {rmse:.4f}")



    # R² (Coefficient of Determination):
    # R² measures how well the predictions explain the variability of the data.
    # Values close to 1.0 are excellent, indicating the model explains most of the variance.
    # Negative values indicate poor fit.
    # Closer to 1.0 is better. Values > 0.7 are generally good; < 0.5 indicates underfitting.
    print(f"R²: {r2:.4f}")

def feature_engineering(data):
    # Extracting basic time-based features
    data['hour'] = data['time'].dt.hour  # Hour of the day
    data['day_of_week'] = data['time'].dt.dayofweek  # Day of the week (0=Monday, 6=Sunday)
    data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)  # Weekend flag

    # Generate lag features (SARIMA benefits from autoregressive terms)
    for lag in range(1, 5):  # Lag features for the past 4 time steps
        data[f'lag_{lag}'] = data['real_consumption'].shift(lag)

    # Generate rolling window statistics (trend and variability capture)
    for window in [3, 6, 12]:  # Windows of size 3, 6, and 12
        data[f'rolling_mean_{window}'] = data['real_consumption'].rolling(window=window).mean()
        data[f'rolling_std_{window}'] = data['real_consumption'].rolling(window=window).std()

    # Generate exponential moving averages (for smoother trends)
    for span in [3, 5]:  # Spans of size 3 and 5
        data[f'ema_{span}'] = data['real_consumption'].ewm(span=span, adjust=False).mean()

    # Capture seasonality with cyclic features for hour
    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)  # Cyclic hour feature (sine)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)  # Cyclic hour feature (cosine)

    # Add a detrended feature (to focus on residual patterns)
    data['detrended'] = data['real_consumption'] - data['real_consumption'].rolling(window=24).mean()

    # Percentage change in consumption (to capture relative variations)
    data['pct_change'] = data['real_consumption'].pct_change()

    # Seasonal difference (important for SARIMA to handle seasonal cycles)
    data['seasonal_diff'] = data['real_consumption'].diff(24)  # Assuming daily seasonality in hourly data

    # Fill NaN values generated by rolling, lag, or pct_change features
    data.fillna(method='bfill', inplace=True)  # Backfill
    data.fillna(method='ffill', inplace=True)  # Forward fill

    return data




In [3]:
df = upload_data()

data_metrics(data=df, real="real_consumption", predicted="predicted_consumption")

df = feature_engineering(df)


MAE: 56.7256
MSE: 5824.3342
RMSE: 76.3173
R²: 0.9176


In [5]:
last_5_months_start = df['time'].max() - pd.DateOffset(months=5)
test_mask = df['time'] >= last_5_months_start

last_5_months_df = df[test_mask]
df = df[~test_mask]



In [6]:
# df = df[['time','real_consumption','predicted_consumption',"ema_5","lag_1","hour","hour_sin","hour_cos"]]

# Step 2: Separate features (X) and target variable (y)
X = df.drop(columns=[ 'time', 'real_consumption', 'predicted_consumption'])  # Features
y = df['real_consumption']  # Target variable


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42, shuffle=True)

In [ ]:
from pmdarima import auto_arima

# Automatically find the best parameters
auto_model = auto_arima(df['real_consumption'],
                        seasonal=True,
                        m=24,  # Adjust 'm' based on seasonality (e.g., 24 for hourly data, 12 for monthly data)
                        trace=True,
                        error_action='ignore',
                        suppress_warnings=True,
                        stepwise=True)

# Display the best parameters
print("Best SARIMA parameters:", auto_model.order, auto_model.seasonal_order)


Performing stepwise search to minimize aic


In [57]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Define and fit the SARIMA model
model = SARIMAX(df['real_consumption'],
                order=auto_model.order,
                seasonal_order=auto_model.seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False)

sarima_model = model.fit(disp=False)

# Print model summary
print(sarima_model.summary())


Fitting 3 folds for each of 2592 candidates, totalling 7776 fits
Best Parameters: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 5, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 10, 'subsample': 0.8}


In [61]:
forecast = sarima_model.forecast(steps=len(test))

# Evaluate the model
train_rmse_xgb = mean_squared_error(y_train, y_pred_train, squared=False)
test_rmse_xgb = mean_squared_error(y_test, y_pred_test, squared=False)
train_r2_xgb = r2_score(y_train, y_pred_train)
test_r2_xgb = r2_score(y_test, y_pred_test)

print(f"XGBoost - Train RMSE: {train_rmse_xgb:.2f}, Test RMSE: {test_rmse_xgb:.2f}")
print(f"XGBoost - Train R²: {train_r2_xgb:.2f}, Test R²: {test_r2_xgb:.2f}")


XGBoost - Train RMSE: 16.83, Test RMSE: 16.97
XGBoost - Train R²: 1.00, Test R²: 1.00


In [62]:
temp_df = last_5_months.head(9)[["time","real_consumption"]]

In [63]:
for i in range(1):  # Adding 3 extra rows
    extra_rows = []
    
    start_time = temp_df["time"].iloc[-1] + timedelta(minutes=3)
    
    extra_rows.append((start_time + timedelta(minutes=3 * i), None))

    # Create a DataFrame for the extra rows
    extra_df = pd.DataFrame(extra_rows, columns=["time", "real_consumption"])
    
    # Combine the original and extra DataFrames
    temp_df = pd.concat([temp_df, extra_df], ignore_index=True)


temp_df

,time,real_consumption
0,2024-06-22 18:54:00,1776.203125
1,2024-06-22 18:57:00,1762.210449
2,2024-06-22 19:00:00,1762.830566
3,2024-06-22 19:03:00,1758.603516
4,2024-06-22 19:06:00,1764.305908
5,2024-06-22 19:09:00,1800.765503
6,2024-06-22 19:12:00,1780.111450
7,2024-06-22 19:15:00,1783.973999
8,2024-06-22 19:18:00,1779.342896
9,2024-06-22 19:21:00,NaN


In [76]:
temp_df = last_5_months.head(9)[["time","real_consumption"]]

for i in range(1,250):
    
    
    for i in range(1):  # Adding 3 extra rows
        extra_rows = []

    start_time = temp_df["time"].iloc[-1] + timedelta(minutes=3)

    extra_rows.append((start_time + timedelta(minutes=3 * i), None))

    # Create a DataFrame for the extra rows
    extra_df = pd.DataFrame(extra_rows, columns=["time", "real_consumption"])

    # Combine the original and extra DataFrames
    temp_df = pd.concat([temp_df, extra_df], ignore_index=True)
    
    
    
    
    
    
    to_predict = feature_engineering(temp_df.copy())[['real_consumption','time',"ema_5","lag_1","hour","hour_sin","hour_cos"]].tail(1)
    to_predict_1 = to_predict.drop(columns=['real_consumption','time','ema_5'])  # Features
    # to_predict_1["ema_5"] = to_predict_1["ema_5"]
    prediction = model.predict(to_predict_1)
    
    results = pd.DataFrame({
        'time' : to_predict["time"],
        'predicted_consumption': prediction
    })
    
    
    temp_df.loc[temp_df['time'] == results["time"].tail(1).values[0], ['real_consumption']] = [results["predicted_consumption"].tail(1).values[0]]
    
    


In [77]:
reuslts_df = temp_df.rename(columns={"real_consumption":"predicted_consumption_new"}).merge(last_5_months[["time","real_consumption","predicted_consumption"]], on="time", how="left").dropna()

In [78]:
# Create an interactive plot using Plotly
fig = px.line(reuslts_df, x='time', y=['real_consumption', 'predicted_consumption','predicted_consumption_new'], title="Interactive Time Series Plot")

# Customize the plot
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Values",
    legend_title="Legend",
    xaxis_rangeslider_visible=True  # Enables the date slicer
)

# Show the plot
fig.show()

In [79]:
data_metrics(data=reuslts_df, real="real_consumption", predicted="predicted_consumption_new")

data_metrics(data=reuslts_df, real="real_consumption", predicted="predicted_consumption")

MAE: 40.1773
MSE: 2442.0872
RMSE: 49.4175
R²: 0.9460
MAE: 113.7736
MSE: 14454.6618
RMSE: 120.2275
R²: 0.6806
